<a href="https://colab.research.google.com/github/steveding1/aws-dataengineering/blob/main/CSV_to_S3_RDS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.   Get urls from 'http://insideairbnb.com/get-the-data/' compare the date in the url.
2.   loop:

*   download
*   csv.gz to S3
*   csv.gz to db(optional)
*   successful info to info and table 'log' (optional)
*   delete(optional)

3.  csv.gz files process and load to db from S3.




In [1]:
!pip install boto3
!pip install requests

     |████████████████████████████████| 132 kB 4.8 MB/s 
     |████████████████████████████████| 79 kB 4.1 MB/s 
     |████████████████████████████████| 8.7 MB 9.6 MB/s 
     |████████████████████████████████| 138 kB 39.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 127 kB 5.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's 

In [20]:
import logging
import boto3
from botocore.exceptions import ClientError
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import configparser
import urllib.request
from dateutil.parser import parse
import psycopg2
import gzip
import datetime

In [3]:
def get_urls(url,ext)->list:
  """
  get all the links end with the {ext}etion from the {url} webpage
  """
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  return [a['href']for a in soup.findAll('a', href=re.compile(ext+"$"))]


In [4]:
def is_date(ds:str)->bool:
  """
  test if a string is a legal date format in yyyy-mm-dd
  """
  if re.match(r'\d{4}-\d{2}-\d{2}', ds):
    try:
        return bool(parse(ds))
    except:
      False
  return False
#is_date('2001-13-30')

In [5]:
def parseurl(url:str)->list:
  """
  get nation city and date from a url string 
  """
  splits = url.split('/')
  try:
    nation = splits[splits.index('data.insideairbnb.com')+1]
  except ValueError:
    print('The url not matches the rule',url)

  for i,item in enumerate(splits):
    if is_date(item):
      date = item
      city = splits[i-1]
      return [nation, city, date]
      break
  return False

#parseurl(urls[1])

In [6]:
def url_tocsv(urls):
  """
  urls list transform and write to a dataframe
  """
  df = pd.DataFrame(columns=["nation", "city", "date", "url"])
  for url in urls:
    url_df=pd.DataFrame([parseurl(url)+[url]])
    url_df.columns=(["nation", "city", "date", "url"])
    df = pd.concat([df,url_df])
  df.to_csv("urls.csv",index=False)
#url_tocsv(urls)

In [7]:
def upload_s3(file_name, bucket, pkey, skey, object_name=None):
    """Upload a file to an S3 bucket
    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """
    session = boto3.Session(
    aws_access_key_id=pkey,
    aws_secret_access_key=skey
)
    # If S3 object_name was not specified, use file_name
    if object_name is None:
      object_name = os.path.basename(file_name)
    s3_client = boto3.client(
        's3',
         aws_access_key_id=pkey,
    aws_secret_access_key=skey)
    try:
      response = s3_client.upload_file(file_name,bucket,object_name)
    except ClientError as e:
      logging.error(e)
      return False
    return True
    # Upload the file
   

In [8]:
def gz_to_db(fullfilename,conf) -> bool:
  '''
  load a csv gz file to postgres db
  '''
  parser = configparser.ConfigParser()
  parser.read(conf)
  sql_host = parser.get("postgres","host")
  sql_user = parser.get("postgres","username")
  sql_password = parser.get("postgres","password")
  sql_db = parser.get("postgres","database")
  sql_table = parser.get("postgres","table")
  load_query = """
  COPY {sql_table} FROM STDIN WITH CSV HEADER DELIMITER AS ',' QUOTE AS '\"' ;
  """.format(
      sql_table=sql_table
  )
  # Create database connection
  sql_connection = psycopg2.connect(
      host=sql_host,
      user=sql_user,
      password=sql_password,
      database=sql_db
  )
  sql_cursor = sql_connection.cursor()
  try:
    with gzip.open(filename=fullfilename, mode='rb') as local_file:
      sql_cursor.copy_expert(sql=load_query, file=local_file)
      sql_connection.commit()
      return True
  except Exception as load_exception:
      sql_connection.rollback()
      return False
      raise Exception("SQL LOAD error: " + str(load_exception)) from load_exception
  # close sql cursor and connection
  sql_cursor.close()
  sql_connection.close()

In [13]:
def down_upload(conf,keepfiles=False,down=True,to_db=False,to_s3=True):
  parser = configparser.ConfigParser()
  parser.read(conf)
  access_key = parser.get("boto_config","access_key")
  secret_key = parser.get("boto_config","secret_key")
  bucket_name = parser.get("boto_config","bucket_name")
  extention = parser.get("website","extention")
  
  downloads = pd.read_csv('urls.csv')
  if not os.path.exists('download'):
    os.makedirs('download')
  for _,download in downloads.iterrows():
    url = download['url']
    fname = download['nation']+'_'+download['city']+extention
    print('processing: ' + fname)
    fullfilename = os.path.join('download', fname)
    if down:
      urllib.request.urlretrieve(url, fullfilename)
    if to_s3:
      if upload_s3(fullfilename, bucket_name, access_key, secret_key):
        with open('success.log','a') as f:
          f.write(url+','+datetime.datetime.now().strftime('%y-%m-%d,%H%M%S')+',s3\n')
    if to_db:
      if gz_to_db(fullfilename,conf):
        with open('success.log','a') as f:
          f.write(url+','+datetime.datetime.now().strftime('%y-%m-%d,%H%M%S')+',db\n')
    if keepfiles is False:
      os.remove(fullfilename)

In [14]:
def main():
  CONF = "conf.conf"
  parser = configparser.ConfigParser()
  parser.read(CONF)
  website = parser.get("website","url")
  extention = parser.get("website","extention")
  urls = get_urls(website,extention)
  url_tocsv(urls)
  down_upload(conf=CONF,keepfiles=True,to_db=True,to_s3=False,down=False)

In [ ]:
if __name__ == "__main__":
  main()

processing: the-netherlands_amsterdam.listings.csv.gz
processing: belgium_antwerp.listings.csv.gz
processing: united-states_asheville.listings.csv.gz
processing: greece_athens.listings.csv.gz
processing: united-states_austin.listings.csv.gz
processing: thailand_bangkok.listings.csv.gz
processing: spain_barcelona.listings.csv.gz
processing: australia_barossa-valley.listings.csv.gz
processing: australia_barwon-south-west-vic.listings.csv.gz
processing: china_beijing.listings.csv.gz
processing: belize_belize.listings.csv.gz
processing: italy_bergamo.listings.csv.gz
processing: germany_berlin.listings.csv.gz
processing: italy_bologna.listings.csv.gz
processing: france_bordeaux.listings.csv.gz
processing: united-states_boston.listings.csv.gz
processing: united-kingdom_bristol.listings.csv.gz
processing: united-states_broward-county.listings.csv.gz
processing: belgium_brussels.listings.csv.gz
processing: argentina_buenos-aires.listings.csv.gz
processing: united-states_cambridge.listings.csv.

In [18]:
parser = configparser.ConfigParser()
parser.read('conf.conf')
sql_host = parser.get("postgres","host")
sql_user = parser.get("postgres","username")
sql_password = parser.get("postgres","password")
sql_db = parser.get("postgres","database")
sql_table = parser.get("postgres","table")

# Create database connection
sql_connection = psycopg2.connect(
    host=sql_host,
    user=sql_user,
    password=sql_password,
    database=sql_db
)
sql_cursor = sql_connection.cursor()
sql_cursor.execute('select * from '+sql_table)
sql_cursor.fetchall()

[]